<a href="https://colab.research.google.com/github/esul-gif/tom-jerry/blob/main/Tom_and_Jerry_ML_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd #data structures and manipulation
from skimage.io import imread #method for loading images
from skimage.feature import hog #feature extraction to identify shape and structure

# Data import

In [ ]:
!unzip -q archive.zip -d /content

[archive.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of archive.zip or
        archive.zip.zip, and cannot find archive.zip.ZIP, period.


In [ ]:
gt = pd.read_csv('ground_truth.csv')
dataset_path = '/content/dataset/Euc'

In [ ]:
from glob import glob
# we list the directories inside the dataset main directory
elements = glob(join(dataset_path, '*'))
print(len(elements), elements)

In [1]:

print('hi')
print('do you see me')

hi
